### Tree Augmentation

1. subtree-size: supports `rank(x)` and `select(i)`.
2. level-linking & subtree-min-max: supports finger-search-property (fast when the target node is close to the given node).
3. tree-on-node: supports orthogonal-range-search.

### problem 5-1

1. For simplicity, we assume that $x.key < k$.
```python
def finger_search(x, k):
    cur = x
    # phase 1: up and right
    while cur.right.key < k:
        cur = cur.right
        cur = cur if cur.up is None else cur.up # go up if possible

    # phase 2: down and right
    while True:
        if cur.right.key > k:
            if cur.down is None:
                return cur
            cur = cur.down  # go down if possible
        else:
            cur = cur.right

    # never reach
    return None
```
The total search path has two phases: we first move up-and-right and then down-and-right. We suppose that the highest level during search is $d = \lg|rank(x.key)-rank(k)| = \lg m$ with high probability and for $x$, we need $d$ heads to get up and for $y$, we also need $d$ heads to get up and converge with $x$.

2. Augmentation: For each node (except bottom nodes), we store its distance to its right neighbor at same level. For `search(x)`, there's no extra cost. For `insert(x)`, when we promote a node, we can update the distance of its left neighbor and calculate its self distance in $O(1)$ each. For `delete(x)`, we can also update all neighbors of its promoted node in $O(1)$ each.

3. 
```python
def rank_search(x, r):
    cur = x
    # phase 1: up and right
    while cur.dist < r:
        d = cur.dist
        cur = cur.right
        cur = cur if cur.up is None else cur.up # go up and right
        r -= d

    # phase 2: down and right
    while True:
        d = cur.dist
        if d > r:
            if cur.down is None:
                return cur
            cur = cur.down  # go down if possible
        else:
            cur = cur.right
            r -= d

    # never reach
    return None
```

### problem 5-2

1. Heap: We maintain a min-heap of size $m$, whose elements are $(value, index)$ pairs. After traverse $P$, we can sort the heap by $index$ to recover $S$. The time complexity is $O(n\log m) + O(m \log m) = O(n \log m)$.

2. Dynamic Programming (with extensions): $dp(i, j, T)$ represents the maximum total value of at most $j$ prizes in first $i$ prizes if the last prize has type $T$.  
$$
\begin{align}
dp(i, j, A) &= \max\{ \\
    & dp(i-1, j, A), \\
    & dp(i-1, j-1, A) + value_j,\ type_j = A \\
\} \\

dp(i, j, B) &= \max\{ \\
    & dp(i-1, j, B), \\
    & dp(i-1, j-1, A) + value_j,\ type_j = B \\
    & dp(i-1, j-1, B) + value_j,\ type_j = B \\
\}
\end{align}
$$
The time complexity is $O(mn)$.

3. Dynamic Programming (with constraints): $dp(i, j)$ represents the maximum total value of at most $j$ prizes ended with $prize_i$.  
$$
\begin{align}
dp(i, j) = \max_{k=0}^{i-1}\{dp(k, j-1)\} + value_i,\ value_k \le value_i
\end{align}
$$
We use an augmented-BST (with maximum value field each node) to support $O(\log n)$-time query and insert. Therefore, the time complexity is $O(mn\log n)$.

4. Dynamic Programming (on tree): $dp(v, j)$ represents the maximum total value of at most $j$ prizes in subtree $v$.  
$$
\begin{align}
dp(v, j) &= \max_{k=0}^{j-1}\{dp(v.left, k) + dp(v.right, j-1-k)\} + value_v \\
dp(v, 0) &= 0
\end{align}
$$
The time complexity is $O(mn^2)$.